<a href="https://colab.research.google.com/github/vivekpenuli/hyperparameter-tuning/blob/master/hyperparameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn import svm, datasets
iris = datasets.load_iris()

In [3]:
iris.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [29]:
import pandas as pd
df = pd.DataFrame(iris.data,columns=iris.feature_names)
df['flower'] = iris.target

df[47:150]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
47,4.6,3.2,1.4,0.2,0
48,5.3,3.7,1.5,0.2,0
49,5.0,3.3,1.4,0.2,0
50,7.0,3.2,4.7,1.4,1
51,6.4,3.2,4.5,1.5,1
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [5]:
import pandas as pd
df = pd.DataFrame(iris.data,columns=iris.feature_names)
df['flower'] = iris.target
df['flower'] = df['flower'].apply(lambda x: iris.target_names[x])
df[47:150]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
47,4.6,3.2,1.4,0.2,setosa
48,5.3,3.7,1.5,0.2,setosa
49,5.0,3.3,1.4,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
51,6.4,3.2,4.5,1.5,versicolor
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


approch 1: change the c value every time and check where you get max score

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.3)

In [24]:
# 1 time
model = svm.SVC(kernel='rbf',C=30,gamma='auto')
model.fit(X_train,y_train)
model.score(X_test, y_test)

0.9777777777777777

In [9]:
# 2 time here we run the train test cell again and then  run this cell and check waht will be accuracy
model = svm.SVC(kernel='linear',C=1,gamma='auto')
model.fit(X_train,y_train)
model.score(X_test, y_test) #very time consuming

0.9777777777777777

here in above case we have to run train test split then model.score for every new c value
but here using cross val score we can do the same without running train test split agian and again

In [10]:
from sklearn.model_selection import cross_val_score

In [11]:
(cross_val_score(svm.SVC(kernel='linear',C=10,gamma='auto'),X_train, y_train, cv=5)).mean()

0.9238095238095239

In [12]:
(cross_val_score(svm.SVC(kernel='rbf',C=10,gamma='auto'),X_train,y_train, cv=5)).mean()

0.9523809523809523

In [14]:
(cross_val_score(svm.SVC(kernel='rbf',C=1,gamma='auto'),X_train,y_train, cv=5)).mean() 

0.9428571428571428

Above approach is tiresome and very manual. We can use for loop as an alternative so that we can do thsi task more fast

In [15]:


kernels = ['rbf', 'linear']
C = [1,10,20]
avg_scores = {}  #it is a dictionary to store the kernal and c with its score
for kval in kernels:  #muiltiple for loop concept
    for cval in C:
        cv_scores = cross_val_score(svm.SVC(kernel=kval,C=cval,gamma='auto'),X_train,y_train, cv=5)
        avg_scores[kval + '_' + str(cval)] = (cv_scores).mean()  

avg_scores
#here instead of using loop cncept we will use grid search to perform the same opertaion

{'linear_1': 0.9714285714285713,
 'linear_10': 0.9238095238095239,
 'linear_20': 0.9428571428571428,
 'rbf_1': 0.9428571428571428,
 'rbf_10': 0.9523809523809523,
 'rbf_20': 0.9428571428571428}

grid search

In [16]:
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(svm.SVC(), {
    'gamma':['auto','scale'],
    'C': [1,10,20  ,30],
    'kernel': ['linear', 'rbf','poly','sigmoid']  # 'precomputed' is not used here
}, cv=5, return_train_score=False)

In [17]:
clf.fit(X_train,y_train)
clf.cv_results_

{'mean_fit_time': array([0.00052743, 0.00053358, 0.000424  , 0.00084209, 0.00044384,
        0.00061789, 0.00046525, 0.00089545, 0.00038867, 0.00045791,
        0.00043812, 0.00082545, 0.00043807, 0.00048084, 0.00046062,
        0.00087857, 0.00039759, 0.00046034, 0.00042005, 0.00082755,
        0.00044708, 0.00047698, 0.00045733, 0.0008769 , 0.00039067,
        0.00046473, 0.00046539, 0.00081816, 0.00044751, 0.0004971 ,
        0.00048313, 0.00087867]),
 'mean_score_time': array([0.00064559, 0.00035868, 0.00026016, 0.00035162, 0.00028734,
        0.00030589, 0.00027795, 0.00034995, 0.00024996, 0.00027966,
        0.00026178, 0.00034342, 0.00025654, 0.00028   , 0.00027437,
        0.00034914, 0.00025473, 0.00027218, 0.00027385, 0.00034328,
        0.00026541, 0.00027094, 0.00026207, 0.00034318, 0.0002522 ,
        0.00027947, 0.00031333, 0.00036302, 0.00026121, 0.00034432,
        0.000283  , 0.00034518]),
 'mean_test_score': array([0.97142857, 0.94285714, 0.96190476, 0.37142857, 0.971

In [18]:
df = pd.DataFrame(clf.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000527,0.000140,0.000646,0.000617,1,auto,linear,"{'C': 1, 'gamma': 'auto', 'kernel': 'linear'}",1.000000,1.000000,1.000000,0.904762,0.952381,0.971429,0.038095,1
1,0.000534,0.000095,0.000359,0.000107,1,auto,rbf,"{'C': 1, 'gamma': 'auto', 'kernel': 'rbf'}",1.000000,1.000000,1.000000,0.904762,0.809524,0.942857,0.076190,17
2,0.000424,0.000011,0.000260,0.000008,1,auto,poly,"{'C': 1, 'gamma': 'auto', 'kernel': 'poly'}",1.000000,1.000000,1.000000,0.857143,0.952381,0.961905,0.055533,3
3,0.000842,0.000034,0.000352,0.000006,1,auto,sigmoid,"{'C': 1, 'gamma': 'auto', 'kernel': 'sigmoid'}",0.380952,0.380952,0.380952,0.380952,0.333333,0.371429,0.019048,25
4,0.000444,0.000014,0.000287,0.000024,1,scale,linear,"{'C': 1, 'gamma': 'scale', 'kernel': 'linear'}",1.000000,1.000000,1.000000,0.904762,0.952381,0.971429,0.038095,1
5,0.000618,0.000095,0.000306,0.000023,1,scale,rbf,"{'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}",1.000000,1.000000,1.000000,0.952381,0.809524,0.952381,0.073771,8
6,0.000465,0.000016,0.000278,0.000019,1,scale,poly,"{'C': 1, 'gamma': 'scale', 'kernel': 'poly'}",1.000000,1.000000,1.000000,0.857143,0.857143,0.942857,0.069985,17
7,0.000895,0.000009,0.000350,0.000007,1,scale,sigmoid,"{'C': 1, 'gamma': 'scale', 'kernel': 'sigmoid'}",0.380952,0.380952,0.380952,0.380952,0.333333,0.371429,0.019048,25
8,0.000389,0.000009,0.000250,0.000001,10,auto,linear,"{'C': 10, 'gamma': 'auto', 'kernel': 'linear'}",0.952381,1.000000,1.000000,0.857143,0.809524,0.923810,0.077372,23
9,0.000458,0.000006,0.000280,0.000010,10,auto,rbf,"{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}",1.000000,1.000000,1.000000,0.857143,0.904762,0.952381,0.060234,8


In [19]:
df[['param_C','param_kernel','mean_test_score']]


,param_C,param_kernel,mean_test_score
0,1,linear,0.971429
1,1,rbf,0.942857
2,1,poly,0.961905
3,1,sigmoid,0.371429
4,1,linear,0.971429
5,1,rbf,0.952381
6,1,poly,0.942857
7,1,sigmoid,0.371429
8,10,linear,0.923810
9,10,rbf,0.952381


In [20]:
print(clf.best_score_)        

print(clf.best_params_)

print(clf.best_estimator_)

# here there is one problem that is there is so many computation has to be done which is time consuming 
#ther fore we will do randamize search

0.9714285714285713
{'C': 1, 'gamma': 'auto', 'kernel': 'linear'}
SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


RandomizedSearchCV

In [21]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(svm.SVC(gamma='auto'), {
        'C': [1,10,20],
        'kernel': ['rbf','linear']
    }, 
    cv=5, 
    return_train_score=False, 
    n_iter=4  # define how many rows we want or howmany combiantion we want
)
rs.fit(X_train,y_train)
pd.DataFrame(rs.cv_results_)[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,10,linear,0.923810
1,20,rbf,0.942857
2,1,linear,0.971429
3,20,linear,0.942857


In [22]:
# examine the best model
print(rs.best_score_)
print(rs.best_params_)

0.9714285714285713
{'kernel': 'linear', 'C': 1}


so here we get to know what is the best parameter for hypertuning 
now


In [28]:
model = svm.SVC(kernel='linear',C=1,gamma='auto')
model.fit(X_train,y_train)
model.score(X_test, y_test)   # what ever done earlier this cell is just to find best parameter
model.predict([[4.6,3.2,1.4,0.2]]) # here it is satosa

array([0])

so now we just did it we know what train,test split do what cross validation do how to use grid serach for model hyperparameter tuning
